In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L2\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['COAST.csv', 'EAST.csv', 'FWEST.csv', 'NCENT.csv', 'NORTH.csv', 'SCENT.csv', 'SOUTH.csv', 'WEST.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[5]))

,Net,Temprature
0,6731.301663,1.3
1,6602.935629,0.8
2,6530.663273,0.6
3,6503.367925,0.8
4,6603.227872,0.9
...,...,...
43818,6453.697286,13.7
43819,6175.763027,14.2
43820,5968.287603,14.0
43821,5769.745220,12.9


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[5]

zoneData = pd.read_csv(os.path.join(path, fileName))
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_SCENT,Temp_SCENT
0,6731.301663,1.3
1,6731.301663,1.3
2,6602.935629,0.8
3,6530.663273,0.6
4,6503.367925,0.8
...,...,...
43819,6453.697286,13.7
43820,6175.763027,14.2
43821,5968.287603,14.0
43822,5769.745220,12.9


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_SCENT,Temp_SCENT,Lag-2,Lag-7
0,7863.816740,3.2,6688.205300,6731.301663
1,7733.428965,1.8,6439.604153,6731.301663
2,7752.625903,0.7,6328.307514,6602.935629
3,7812.730198,-0.3,6354.726927,6530.663273
4,7987.837391,-1.2,6453.118126,6503.367925
...,...,...,...,...
43651,6453.697286,13.7,6210.557281,5780.380475
43652,6175.763027,14.2,6207.856531,5693.923783
43653,5968.287603,14.0,6140.580238,5624.696898
43654,5769.745220,12.9,5939.445038,5512.343425


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 23s 16ms/step - loss: 0.0087 - mape: 3061.3716 - mae: 0.0699 - val_loss: 0.0076 - val_mape: 33.6999 - val_mae: 0.0692
Epoch 2/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0045 - mape: 1940.9948 - mae: 0.0511 - val_loss: 0.0187 - val_mape: 39.6364 - val_mae: 0.1009
Epoch 3/100
1105/1105 [==============================] - 16s 15ms/step - loss: 0.0030 - mape: 1705.3773 - mae: 0.0419 - val_loss: 0.0187 - val_mape: 47.9331 - val_mae: 0.1054
Epoch 4/100
1105/1105 [==============================] - 16s 14ms/step - loss: 0.0028 - mape: 1626.7726 - mae: 0.0403 - val_loss: 0.0152 - val_mape: 35.2620 - val_mae: 0.0939
Epoch 5/100
1105/1105 [==============================] - 15s 14ms/step - loss: 0.0026 - mape: 1396.1748 - mae: 0.0388 - val_loss: 0.0181 - val_mape: 39.2424 - val_mae: 0.0985
Epoch 6/100
1105/1105 [==============================] - 16s 14ms/step - loss: 0.0024 - mape: 1309.3586 - mae: 0.0372 - val_l

Epoch 48/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0011 - mape: 981.0356 - mae: 0.0245 - val_loss: 0.0162 - val_mape: 34.4115 - val_mae: 0.0940
Epoch 49/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0011 - mape: 711.0790 - mae: 0.0246 - val_loss: 0.0189 - val_mape: 35.4117 - val_mae: 0.1007
Epoch 50/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0010 - mape: 597.2659 - mae: 0.0244 - val_loss: 0.0172 - val_mape: 35.4384 - val_mae: 0.0980
Epoch 51/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0010 - mape: 766.4883 - mae: 0.0243 - val_loss: 0.0170 - val_mape: 34.1981 - val_mae: 0.0961
Epoch 52/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0010 - mape: 765.9509 - mae: 0.0241 - val_loss: 0.0180 - val_mape: 36.4651 - val_mae: 0.1003
Epoch 53/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0010 - mape: 940.3752 - mae: 0.0243 - val_l

Epoch 94/100
1105/1105 [==============================] - 17s 16ms/step - loss: 7.2516e-04 - mape: 781.6058 - mae: 0.0202 - val_loss: 0.0187 - val_mape: 36.5249 - val_mae: 0.1013
Epoch 95/100
1105/1105 [==============================] - 17s 16ms/step - loss: 7.0436e-04 - mape: 648.9227 - mae: 0.0199 - val_loss: 0.0165 - val_mape: 35.7362 - val_mae: 0.0973
Epoch 96/100
1105/1105 [==============================] - 17s 16ms/step - loss: 7.0830e-04 - mape: 923.0394 - mae: 0.0200 - val_loss: 0.0187 - val_mape: 37.6660 - val_mae: 0.1022
Epoch 97/100
1105/1105 [==============================] - 17s 16ms/step - loss: 7.0504e-04 - mape: 880.3671 - mae: 0.0199 - val_loss: 0.0179 - val_mape: 37.2535 - val_mae: 0.1006
Epoch 98/100
1105/1105 [==============================] - 18s 16ms/step - loss: 6.9927e-04 - mape: 969.3981 - mae: 0.0198 - val_loss: 0.0174 - val_mape: 36.7080 - val_mae: 0.0986
Epoch 99/100
1105/1105 [==============================] - 17s 16ms/step - loss: 6.8918e-04 - mape: 628.07

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 1804.254863023758


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 16ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.10821219, 0.08045383, 0.06673777, ..., 0.33825195, 0.2945911 ,
       0.24975957], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.26600104, 0.242116  , 0.22642119, ..., 0.22142303, 0.2025681 ,
       0.18779946])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,6437.694035,4776.177734
1,6186.184567,4483.882324
2,6020.918290,4339.452148
3,6036.897519,4289.166992
4,6172.562026,4421.524902
...,...,...
4363,6453.697286,7496.837402
4364,6175.763027,7450.188477
4365,5968.287603,7198.495605
4366,5769.745220,6738.747559


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

13.17602294261721